In [ ]:
try:
    !pip install pyspark=="2.4.5" --quiet
    !pip install pandas=="1.0.4" --quiet
except:
    print("Running throw py file.")

In [1]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark import SparkFiles
from pyspark.sql.types import StringType
import pyspark
import json
import pandas as pd
import time

In [2]:
spark = SparkSession\
        .builder\
        .appName("Desafio Data Engineer Escale - Fabio Kfouri")\
        .getOrCreate()
spark

In [3]:
import os

dataPath = 'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/'

if 'E:\\' in os.getcwd() and 'dataEngineerTest_Escale' in os.getcwd():
    dataPath = os.getcwd() + "/data/"

print(dataPath)


E:\Projetos\Jobs\dataEngineerTest_Escale/data/


In [4]:
question1 = {}
filename = ''

In [5]:
for i in range(9):
    spark.sparkContext.addFile(dataPath + 'part-0000' + str(i) +'.json.gz')

In [6]:
time_limit = (30*60)*1000
print(time_limit)

1800000


In [7]:
def readFile(i):
    filename = 'part-0000' + str(i) +'.json.gz'
    print(filename)
    df = spark.read.json(SparkFiles.get(filename))
    overCategory = Window.partitionBy('anonymous_id','device_family','os_family').orderBy('device_sent_timestamp')
    df = df.withColumn("lag", F.lag('device_sent_timestamp', 1).over(overCategory))
    df = df.withColumn('delta', F.col('device_sent_timestamp') - F.col('lag'))\
        .withColumn('same_section', (F.col('device_sent_timestamp') - F.col('lag')) < time_limit) \
        .withColumn('event_time', (F.col('device_sent_timestamp')/1000).cast('timestamp'))
    df.createOrReplaceTempView("clickstream")
    dados = spark.sql("""
        with table_temp as (--
            SELECT anonymous_id, browser_family, device_family, device_sent_timestamp, event, event_time, n, os_family, platform, NVL(same_section,false) same_section, version
            FROM clickstream t --
            WHERE NVL(same_section, false) = false --remove as sessoes abertas
        ), table_session_id as (--
            SELECT t.*, 'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp )  || '_' ||  anonymous_id as session_id
            FROM table_temp t --
        )
        SELECT COUNT(session_id)  qtd_session
        FROM table_session_id
        """)
    question1[filename] = dados.collect()[0]['qtd_session']

In [8]:
for i in range(9):
    readFile(i)

In [9]:
print(json.dumps(question1))

{"part-00000.json.gz": 10234891, "part-00001.json.gz": 10230776, "part-00002.json.gz": 10227212, "part-00003.json.gz": 10233329, "part-00004.json.gz": 10233306, "part-00005.json.gz": 10235205, "part-00006.json.gz": 10232658, "part-00007.json.gz": 10236335, "part-00008.json.gz": 10228150}


In [10]:
print(question1)

{'part-00000.json.gz': 10234891, 'part-00001.json.gz': 10230776, 'part-00002.json.gz': 10227212, 'part-00003.json.gz': 10233329, 'part-00004.json.gz': 10233306, 'part-00005.json.gz': 10235205, 'part-00006.json.gz': 10232658, 'part-00007.json.gz': 10236335, 'part-00008.json.gz': 10228150}


In [ ]:
def create_

In [ ]:
df0 = readFile(0)
df0.show(3)

In [ ]:
df0.printSchema()

In [ ]:
overCategory = Window.partitionBy('anonymous_id','device_family','os_family').orderBy('device_sent_timestamp')

In [ ]:
df0 = df0.withColumn("lag", F.lag('device_sent_timestamp', 1).over(overCategory))

In [ ]:
time_limit = (30*60)*1000
print(time_limit)

In [ ]:
df0 = df0.withColumn('delta', F.col('device_sent_timestamp') - F.col('lag'))\
        .withColumn('same_section', (F.col('device_sent_timestamp') - F.col('lag')) < time_limit) \
        .withColumn('event_time', (F.col('device_sent_timestamp')/1000).cast('timestamp'))

df0.show()

In [ ]:
df0.createOrReplaceTempView("clickstream")

In [ ]:
df_question_1 = spark.sql("""
with table_temp as (--
    SELECT anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, platform, NVL(same_section,false) same_section, version
    FROM clickstream t --
    WHERE NVL(same_section, false) = false --remove as sessoes abertas
), table_session_id as (--
    SELECT t.*, 'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp )  || '_' ||  anonymous_id as session_id
    FROM table_temp t --
)
SELECT COUNT(session_id)  qtd_session
FROM table_session_id
""")
df_question_1.show(300, False)

In [ ]:
dados = df_question_1.toPandas()


In [ ]:
print(dados['qtd_session'])

In [ ]:
x = dados['qtd_session']
x

In [ ]:
x = df_question_1.collect()[0]['qtd_session']
x